# SQL-Driven Recipe Suggestions
### By Erica Laidler

## Introduction
In this project, I build a recipe suggestion system. The system will enable a user to input a variety of requests, and print the recipes which best satisfy the conditions, in descending order. This project involves the application of natural language processing, SQLite, and cosine similarity.

The chosen data set features 4,656 recipes from a variety of sources, with diverse cuisine from all over the world. It includes frecipes from Italy, Sri Lanka, Thailand, Mexico, Croatia, the US, England, and Morocco. It also documents the ingredients for each recipe. The data set is available for public use on my Github, at the link https://github.com/elaidler7/Recipe-Suggestor/blob/main/df_recipes.csv. It is originally from Jack Leitch's GitHub (https://github.com/jackmleitch/Whatscooking-).

I begin by importing the dataset and preprocessing. This involved stripping unnecessary measurements, descriptive words, and punctuation from the ingredients column. After preprocessing thoroughly, I use a SQLite-based system which subsets the data based on user input. The user can choose mandatory ingredients (ingredients which *must* present in the recommended recipes), unwanted ingredients (ingredients which must *not* be in the recipe), and optional ingredients (ingredients that users would *like* to be in the recipe, but aren't required). This allows the user specificy and flexibility in their search results.

I also created another column in the database which stores binary vectors for each recipe. In these vectors, each element corresponds to an ingredient. If a particular ingredient is present in the recipe, the vector has a value of 1 at the corresponding index; otherwise, it has a value of 0.

With SQLite, the data is filtered to include only the recipes that contain the user's required ingredients, and omit their unwanted ingredients. When the user inputs their request, a vector is generated to represent both the required and optional ingredients. The vector is then compared with the vectors associated with recipes meeting the user's non-negotiable  requirements using cosine similarity.

Finally, the recipes are displayed to the user. Based on the original SQLite query, this list include recipes containing the required ingredients and excluding the unwanted ingredients. Additionally, the list is sorted based on the cosine similarity calculation, placing recipes with optional ingredients and as few additional ingredients as possible closer to the top. The user thus sees a list of recipes which align with their requirements, sorted in such a way that the first results are most likely to be achievable given the ingredients they have in their kitchen.

Below, I will explain each step of the project with accompanying code.

# Data Import

The first step is to import all relevant packages.

In [2]:
import sqlite3
import pandas as pd
import os
import re
import spacy
nlp = spacy.load("en_core_web_sm")
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import inflect
import numpy as np
from scipy.sparse import csr_matrix
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


The next step is to import the recipes dataset and examine it.

In [3]:
# Import the dataset
url = "https://raw.githubusercontent.com/elaidler7/Recipe-Suggestor/5531a487906c3e4b78ce55873d3cd2ca29a0102f/df_recipes.csv"

# Use the data from the CSV file to create a DataFrame
recipes_df = pd.read_csv(url)

# Take a look at the dataset
recipes_df

,recipe_urls,recipe_name,ingredients
0,https://www.jamieoliver.com/recipes/duck-recip...,Roast duck with Marsala gravy,"['1 x 1.6kg whole duck', '2 heaped teaspoons C..."
1,https://www.jamieoliver.com/recipes/vegetable-...,Best-ever Brussels sprouts,"['800 g Brussels sprouts', '2 higher-welfare C..."
2,https://www.jamieoliver.com/recipes/pasta-reci...,Beautiful courgette carbonara,"['6 medium green and yellow courgettes', '500 ..."
3,https://www.jamieoliver.com/recipes/vegetable-...,Roasted black bean burgers,"['1½ red onions', '200 g mixed mushrooms', '10..."
4,https://www.jamieoliver.com/recipes/chicken-re...,Chicken & tofu noodle soup,"['2 shallots', '2 cloves of garlic', '2 cm pie..."
...,...,...,...
4642,https://www.allrecipes.com/recipe/213809/thai-...,Thai-Style Steamed Pumpkin Cake Recipe - Allre...,"[""1 (2 pound) pumpkin - peeled, seeded, and gr..."
4643,https://www.allrecipes.com/recipe/41751/delici...,Delicious Spicy Tomato Salad Recipe - Allrecip...,"[""1 apple - peeled, cored and sliced"",""1 chile..."
4644,https://www.allrecipes.com/recipe/264507/spicy...,Spicy Sriracha Meatballs Recipe - Allrecipes.com,"[""1 egg"",""1 lime, zested"",""1 tablespoon creamy..."
4645,https://www.allrecipes.com/recipe/264012/ajad-...,Ajad (Authentic Thai Cucumber Salad) Recipe - ...,"[""1 1/2 teaspoons salt"",""1/2 cup white vinegar..."


# Preprocessing the Data



As shown above, the DataFrame contains 4,647 recipes, with columns to store the URL, recipe name, and ingredients for each. The ingredients column, however, requires significant processing. Though each element appears to be a list, they are actually strings, which makes them a challenge to work with. Also, there are a large number of descriptive and measurement words, which will not be useful. For the purposes of this project, I don't need to know that a recipe calls for 200 grams of mixed mushrooms, for example. It is sufficient to know simply that it requires mushrooms. Therefore, I will perform a variety of preprocessing steps to this column and create a new column with cleaned and organized data.

The following functions, 'spaces_present', 'reformat', and 'string_to_list' are helper functions that are used in 'clean_items', the main function which will perform the majoirty of the preprocessing steps.

The first function, 'spaces_present', checks to see if a row of the ingredients column is in a certain form. I noticed that not all rows are formatted the same way. Some of the strings have spaces between comma-separated values (ingredients), and some do not. This is important to know because when converting the strings into lists, the 'clean_items' function needs to know whether there are spaces between the values, and treat each case accordingly.

In [4]:
def spaces_present(input_string: str) -> bool:
    """
    Checks to see if comma-separated values in the string are separated by spaces.

    Parameters:
    input_string (str): The string to be checked.

    Returns:
    bool: True if spaces are present, False if no spaces present.
    """
    # Set default value of boolean to false.
    spaces_present_bool = False

    # Remove brackets (takes substring without brackets) and split this into substrings (each a comma-separated value, or a single ingredient).
    # Note that the string may use '' or "", and the code accomodates either case.
    if '"' in input_string:
        ingredients = input_string[1:-1].split('",')
    else:
        ingredients = input_string[1:-1].split("',")

    for i in range(len(ingredients) - 1):
        # Set ingredient 1 (one comma-separated value).
        ingredient1 = ingredients[i]
        # Set ingredient 2.
        ingredient2 = ingredients[i + 1]
        # If there is a space between the ingredients, change the boolean to be True. Otherwise, it remains false.
        if ' ' in ingredient1[-1] or ' ' in ingredient2[0]:
            spaces_present_bool = True
    return(spaces_present_bool)


The next function is 'reformat.' As stated previously, the rows of the ingredients column are typically in one of two formats. Some include a space between ingredients, while others do not. The 'clean_items' function calls on 'spaces_present' to check if there are spaces between the ingredients for a particular row, and if there are not, the function calls on 'reformat' to convert the string into the format with spaces. In this way, it is possible to standardize the rows, so that the 'clean_items' function can handle them in the same way.

The 'reformat' function also calls on the helper function, 'add_single_quotes,' which replaces any double quotes with single quotes, and further ensures that all rows end up in the same form.

In [5]:
def add_single_quotes(input_string: str) -> str:
    """
    Replaces the double quotes with single quotes in a string.

    Parameters:
    input_string (str): The string to be adjusted.

    Returns:
    str: Returns a string with the changes applied.
    """
    # Replace double quotes with single quotes
    input_string = input_string.replace('"', "'")
    return input_string

def reformat(input_string: str) -> str:
    """
    Reformats a string by replacing double quotes with single quotes, and adding spaces between comma-separated values

    Parameters:
    input_string (str): The string to be reformatted

    Returns:
    str: The reformatted string
    """
    # Remove the brackets and split the string by commas.
    items = input_string[1:-1].split(',')

    # Reformat: Join the items with (', ') instead of the (",") (add space).
    space_added = ', '.join(item.strip() for item in items)

    # Reformat: Change the double quotes into single quotes by calling on the helper function.
    punctuation_added = add_single_quotes(space_added)

    # Add the brackets back in, now that the changes have been made.
    final_format = '[' + punctuation_added + ']'

    return final_format

After 'clean_items' calls on 'reformat' to ensure that all rows in the ingredients column are structured in the same way, the next step is to convert them all to lists. This will make them easier to work with.

Originally, the strings are structured in the form "['2 tomatoes', '3 lb potatoes',...]." After applying 'string_to_list', they will instead take the form of a list, for which each ingredient (i.e. "2 tomatoes") is an element in the list.

In the code below, I also display the results of 'string_to_list' on one particular row of the ingredients column, to serve as an example.

In [6]:
def string_to_list(text: str) -> list:
    """
    Takes a string of the form "['xyz', 'abc',...]" and returns a list, containing the extracted comma-separated
    values in the string.
    Note that the original string must be in the correct form in order for the function to work as intended.

    Parameters: The string to be converted into a list.

    Returns:
    list: The list version of the inputted string.
    """
    # Replace unwanted characters like "['", "']", "', '", and "',".
    cleaned_text = text.replace("['", "").replace("']", "").replace("', '", "',").replace("',", ",").replace('["', '').replace('"]', "").replace('", "', "',").replace('",', ",")

    # Split into list of substrings (using ',' as delimeter).
    cleaned_text2 = cleaned_text.split(',')

    # Strip leading and trailing whitespace. Also eliminates any substring "", because it only includes items for
    #which item.strip() does not return an empty string.
    ingredients = [item.strip() for item in cleaned_text2 if item.strip()]

    #ingredients = [item.strip().strip("'") for item in cleaned_text3 if item.strip().strip("'")]
    return ingredients

# Test it out.
ingredient_string = "['1 x 1.6kg whole duck', '2 heaped teaspoons Chinese five-spice powder', '1 clementine', '6 fresh bay leaves', 'GRAVY', '', '1 bulb of garlic', '2 carrots', '2 red onions', '3 tablespoons plain flour', '100 ml Marsala', '1 litre organic chicken stock']"
ingredient_list = string_to_list(ingredient_string)
print(ingredient_list)


['1 x 1.6kg whole duck', '2 heaped teaspoons Chinese five-spice powder', '1 clementine', '6 fresh bay leaves', 'GRAVY', '1 bulb of garlic', '2 carrots', '2 red onions', '3 tablespoons plain flour', '100 ml Marsala', '1 litre organic chicken stock']


The final step before implementing the 'clean_items' function is to develop a thorough list of words which can be considered unnecessary, and hence removed.

One of the main unnecessary categories is words which pertain to measurements. This includes words like "cup", "tablespoon", and "quart", as shown in the 'measurement_words' list.

Also unnecessary, in most cases, are descriptive words like "roasted," "tightly," and "quality." For instance, it is not useful to retain that the recipe calls for "quality tomatoes." "Quality" is subjective and not useful; simply "tomatoes" is preferred.

Most phrases are also not crucial to include. If phrases are not removed, there runs the risk of the program treating it as an ingredient, as it is technically a comma-separated value. For instance, the recipe might call for "3 heads of broccoli, stalks removed." For this reason, I specifically removed phrases like "stalks removed", "extra for greasing," etc.

I also removed traditional stop words, like "of", "to", and "the," words which occur frequently and do not provide any information about the ingredients.

Determining which words to remove is a complex process. The choices are not objective, because a word may provide vital information in some cases and be superfluous in others. For instance, the "whole" in "whole grain bread" specifies the type of bread, while it is not as necessary in "1 whole pineapple". But as there are over 4,000 recipes, it is not time efficient to manually parse through all cases in which a word was used.

My process for gathering these lists of words derived from a combination of research, manual assessment of the data set, and common sense. Some of the recipes are from the same cooking site, and certain phrases, like "depending on your preference," are commonly used across recipes of the same source. Overall, in the process of creating a list of words to remove, I sought to balance time efficiency with ensuring the elimination of the most unnecessary words possible.

In [7]:
measurement_words = ["cup", "tablespoons", "teaspoons", "tsp", "quart", "ounce", "oz", "pound", "lb", "stick", "gram", "g", "milliliter", "ml", "liter", "litre", "litres", "gallon", "bunch", "sprig", "cm", "piece", "star", "mug", "kg", ".kg", "kilogram", "jar", "rashers",  "handful", "stalks", "drizzle", "head", "splash", "pinch", "bottle"]
description_words = [
    "fresh", "roasted", "mixed", "ground", "soft", "ripe", "heaped", "medium",
    "plus extra to serve", "plain", "whole", "heaping", "debearded",
    "from sustainable sources", "glass", "free-range", "a good drizzle",
    "plus extra for drizzling", "to cook", "bone in", "skin on",
    "small", "prepared", "slice", "pints", "cans", "large", "knob", "such as", "swig", "ounces", "crumbled", "scaled and pin boned", "optional", "bulb", "spears",
    "whole", "can", "tin", "quality", "depending on your preference", "plus", "scaled and gutted",
    "sticks", "piece", "fluid", "shaving", "tightly", "long", "dry", "raw",
    "package", "-", "()", "containers", "cut into florets just larger than", "packages", "sinews removed", "segmented", "cups", "sheets", "slices",  "such as", "few", "optional: ", "handfuls",
    "knobs", "chopped", "pounds", "inch", "sprigs", "bunches", "leftover", "[", "/", "\\", "\u00BD", "\u00BC", "\u2153", "\u00BE", "%" # Represents Unicode ½, ¼ ⅓, ¾
]
exclude_words= ["skinless", ".", "taste", "thumb-sized", "(g)", "herbs(", "tablespoon",  "ml-", "vac-packed", "stale",
                "teaspoon", "etc", "sides", "filleted",  "each","skinned", "(cold)", "extra", "cleaned",  "trimmed", "sliced",
                "grated", "moist", "jacob's", "-", "cold", "interesting", "roughly", "chilled", "softened",  "serve",  "melted",  "jointed", "(.kg)", "scaled",
                "pin-boned", "(optional)", "freshly", "use"
                 ]
phrases_to_remove = ["(.cm dice)", "(see tip)", "larger ones halved", "for frying", "peeled chipped",
                    "or any other floury potatoes thickly diced", "in place of", "juice of", "gutted (kg total)", "for brushing over pastry",
                    "boneless", 'topping', "extra for greasing", "about", "big handful fine", "coarsely", "for the pie", "bone in", "peeled but with tails lefts on", "extra if needed", "on vine", "for deep frying",
                    "deseeded sliced", "shredded", "stalks removed", "lightly toasted", "thickly", "cut into 5cm lengths", "cut into big chunks", "trimmed cut into pieces", "gutted scaled",  "with shells",
                    "cut into florets just larger than potato", "filleted (ask your fishmonger to do this)",
                    "(g per bird_", "sinews removed", "on vine", "giblets removed", "dried", "(depending on age appetite your children)", "for deep frying", "such as dill",
                    "different roughly", "you might only need", "as they tend to be larger", "to cook",  "you will need", "membrane removed", "for paste", "for rad rum bbq sauce",
                    "(available from specialist stores)", "pestle and mortar", "to serve", "colours if possible", "you will need", "skin removed",
                    "you also need", "torn into chunks", "a few sprigs left whole", ")", "topped and tailed", "if out of season", "broken into leaves", "(", "cut into chunks (ask your butcher)", "beaten, finely chopped, or mix both",
                    "leaves picked", "cut into wide strips", "torn into pieces", "cubed", "very", "squeeze", "deboned and diced about 450g meat", "for salsa verde:", "drained", "(g)", 'to serve', "made using eggs",
                    "extra to serve", "basic risotto recipe", "pinboned", "preferably scottish", "with giblets", "for grating", "peeled", "''", "drizzling", "(see tip)" "ml–", "deseeded",
                    "for grating", "excess fat", "sprinkling", "level", "optional" "leaves reserved to garnish", "cut into pieces", "washed", "finely", "for dusting", "ones ripped up", "approximately", "beaten", "skin on", "skin off", "from sustainable sources", "deboned (your fishmonger can do this for you",
                     "on vine, )", "broken into florets", "quartered", "undyed", "from the heart", "shots", "french all fat removed", "cored", "diced", "rinsed", "scored", "into rings", "cut into thin wedges", "extra for decorating", "about tubes", "at room temperature", "christmas turkey with homemade gravy",
                     "cut into chunks", "cooked", "cut into 2cm chunks", "plus extra for serving", "bashed", "tails left on", "around", "cut into wedges", "washed and spun dry", "cut in half horizontally", "cut into quarters", "halved", "on the vine", "ask your fishmonger", "other leaves", "such as", "such as camargue",
                     "made using free-range eggs", "scrubbed clean", "thinly", "crushed", "outer leaves torn off", "cut into 4 wedges", "butterflied with tail-shells on", "ones up", "good", "preferably organic", "brushing over pastry", "if you can get it", "(neck end only)", "ml–", "leaves picked", "if you want to pimp your beans",
                     "thick", "canned", "thin", "smoked", "loaf", "organic", "tbsp", "pinch", "split", "torn", "seeded", "fine", "toasted", "light", "cube", "box", "piece", "drop", "refrigerated", "big", "shelled", "broken", "stem", "lean", "packed", "pitted", "firm", "tinned", "pint", "preferably", "quart", "pod", "tsapoon",
                     "bulk", "carton", "lengthwise", "goya", "soaked", "lightly", "marinated", "ball", "bottled", "envelope", "chunky", "wineglass", "shell", "stewing", "progressor", "top", "deveined", "needed", "flaked", "desired", "powdered", "brewed", "mashed", "strip", "woody", "shoulder", "lug", "block", "generous",
                     "additional", "julienned", "pre", "shaved", "cumberland", "deboned", "colour", "blanched", "cooking", "assorted", "slivered", "serving", "sifted", "casing", "grilled", "jaggery", "recipe", "fully", "reduced", "garnish", "live", "stalk", "rolled", "loosely", "tub", "really", "old", "picked",
                     "brushing", "homemade", "snipped", "separated", "edible", "cook", "real", "slider", "sturdy", "hungarian", "rinsing", "tempering", "blended", "stiffly", "dust", "xkg", "diameter", "prefer", "active", "lacinato", "wear", "garnishing", "tablspoon", "single", "multicolored", "back", "lengthways", "swasonr",
                     "spreadable", "stemmed", "favorite", "royal", "different", "inch", "aged", "skewer", "natural", "mini", "amount", "basic", "diagonally", "cooled", "crosswise", "bulb", "canning", "hellman", "end", "seasonal", "sheet", "dab", "sustainably", "granny", "grind", "mature", "wrapped", "ring", "skim", "country",
                     "total", "ready", "hidden", "warmed", "stuffed", "smashed", "wood", "colourful", "smooth", "major", "fermented", "destoned", "clarified", "divided", "sectioned", "layer", "optional", "partially", "new", "sharp", "petite", "ideally", "hulled", "flavoured", "ripped", "curly", "shucked", "lukewarm", "made", "across",
                     "whatever", "couple", "stewed", "snapped", "part", "pouch", "day", "cockle", "tenderstem", "johnsonville", "zested", "juiced", "whisked", "unflavored", "depending", "side", "three", "clean", "basket", "johnsonviller", "dressing", "roasting", "pink", "imitation", "lingham", "tart", "make", "individual", "superfine",
                     "process", "summer", "shaving", "tip", "vin", "inner", "nutritional", "log", "horizontally", "sandwich", "pressed", "topped", "eating", "body", "cap", "rubbing", "wheel", "mostly", "tsonr", "granular", "rump", "sauteing", "tabelspoon", "dressed", "distilled", "scoop", "pale", "slit", "patted", "nice",
                     "chipped", "hunk", "dehydrated", "farmer", "philadelphia", "certified", "jacob", "skinny", "sustainable", "tablepoon", "left", "scrubbed", "teaspoon", "rolling", "tore", "united", "pulled", "almost", "desiccated", "xg", "dice", "texturized", "micro", "poached", "belly", "linked", "cooky", "recado", "pata",
                     "charlotte", "differnetly", "size", "flavored", "appetizer", "grating", "diet", "quick", "work", "topside", "rehydrated", "tossed", "fatty", "pit", "tied", "toothpick", "peter", "reconsituted", "hearty", "either", "unsifted", "stir", "pickling", "appenzeller", "nature", "teasponn", "progressotm", "giardiniera",
                     "enough", 'pounded', 'tesapoon', "savory", "unpopped", "greaing", "baked", "crisp", "holy", "frying", "triple", "choice", "tabelspoons", "remaining", "filled", "stoned", "cavity", "sized", "unshelled", "length", "discardin", "making", "equivalent", "possible", "gentelman", "ratte", "teaspogar", "decorating", "foot",
                     "well", 'barillar', "stirred", "disk", "floury", "splash", "juicy", "tablepsoons", "second", "closed", "fashion", 'shredded', 'boneless',  'pared',  'leaf',  'container', 'removed',  'seasoned', 'ounce'

]

stop_words = ["of", "to", "the", "and", "x", "a", "an", "for", "or", "in"]

Finally, I implement the 'clean_items' function. This function is intended to take a single row from the 'ingredients' column of the 'recipes_df' DataFrame. This function takes the string and removes tokens that do not contribute greatly to the meaning of the text, at least in the context of an ingredient list. First, it removes stop words ('of', 'the', and 'and'). These do not contribute much semantic meaning to the list of ingredients. It also removes measuring words like 'tablespoon', 'cup', and 'teaspoon, which describe the quantity of the food rather than the food itself. It also converts the text from a string into a list and ensures that each ingredient is formatted correctly.

In [8]:
def clean_items(ingredient_list_og: str) -> list:
    """
    Takes a row of the 'ingredients' column of 'recipes_df' and returns a list, formatted properly and with
    unnecessary words removed

    Parameters: The string to be cleaned (row of 'ingredients' column; represents the ingredients in a recipe)

    Returns: Cleaned ingredient list
    """

    # The elements of the dataset are typically of one of two formats. If it is in the format without spaces, I
    # convert it to the format with spaces by using the add_spaces function.
    if spaces_present(ingredient_list_og) == False:
        ingredient_list_og = reformat(ingredient_list_og)

    # Convert to lower case
    ingredient_list_og2 = ingredient_list_og.lower()

    # Convert string to list using the 'string_to_list' helper function
    ingredient_list = string_to_list(ingredient_list_og2)

    # Initialize cleaned list to put ingredients in after they are cleaned
    cleaned_list = []

    # Iterate over all ingredients in the ingredient list
    for i in range(0, len(ingredient_list)):

        # Set the current ingredient (one of the elements of the ingredient list)
        cur_ingredient = ingredient_list[i]

        # Remove any unwanted phrases (n-grams)
        for phrase in phrases_to_remove:
            cur_ingredient = cur_ingredient.replace(phrase, "")

        # Temporarily split the ingredient string into a list of the individual words within the string
        cur_ingredient = cur_ingredient.split()

        # Remove numbers
        cleaned_ingredient1 = ["".join(filter(lambda char: not char.isdigit(), item)) for item in cur_ingredient]

        # Remove traditional stop words
        cleaned_ingredient2 = [item for item in cleaned_ingredient1 if item not in stop_words]

        # Remove measuring words like 'tablespoon', 'cup', 'teaspoon', etc.
        cleaned_ingredient3 = [item for item in cleaned_ingredient2 if item not in measurement_words]

        # Remove other descriptive words
        cleaned_ingredient4 = [item for item in cleaned_ingredient3 if item not in description_words]

        # Remove any words in the exclude_words list defined earlier
        cleaned_ingredient5 = [word for word in cleaned_ingredient4 if word.strip() and not any(exclude in word for exclude in exclude_words)]

        # Concatenate the elements of the list back into a string
        cleaned_ingredient_string1 = ' '.join(cleaned_ingredient5)

        # Remove any leading whitespace
        cleaned_ingredient_string = cleaned_ingredient_string1.lstrip()

        # As long as it is not an empty list after cleaning, append the ingredient to the cleaned ingredients list
        if cleaned_ingredient_string != "":
            cleaned_list.append(cleaned_ingredient_string)

    return cleaned_list

# Test it out
cleaned_items1 = clean_items(recipes_df['ingredients'][2])
cleaned_items2 = clean_items(recipes_df['ingredients'][3])
print("The first original list is as follows :" + recipes_df['ingredients'][2] + ".")
print("The second list, after cleaning, is " + str(cleaned_items1) + ".")
print("   ")
print("The second original list is as follows :" + recipes_df['ingredients'][3] + ".")
print("The first list, after cleaning, is " + str(cleaned_items2) + ".")

The first original list is as follows :['6 medium green and yellow courgettes', '500 g penne', '4 large free-range eggs', '100 ml single cream', '1 small handful of Parmesan cheese', 'olive oil', '6 slices of higher-welfare back bacon', '½ a bunch of fresh thyme , (15g)', 'a few courgette flowers , (optional)'].
The second list, after cleaning, is ['green yellow courgettes', 'penne', 'eggs', 'cream', 'parmesan cheese', 'o oil', 'bacon', 'thyme', 'courgette flowers'].
   
The second original list is as follows :['1½ red onions', '200 g mixed mushrooms', '100 g rye bread', 'ground coriander', '1 x 400 g tin of black beans', 'olive oil', '40 g mature Cheddar cheese', '4 soft rolls', '100 g ripe cherry tomatoes', '1 lime', 'chipotle Tabasco sauce', '1 ripe mango', '1 ripe avocado', '4 tablespoons natural yoghurt', '4 sprigs of fresh coriander'].
The first list, after cleaning, is ['red onions', 'mushrooms', 'rye bread', 'coriander', 'black beans', 'o oil', 'cheddar cheese', 'rolls', 'cherr

Now I add a column to the original data frame (cleaned_ingredients) that contains the tidied up version of the ingredients column.

In [9]:
# Apply the clean_items function to the 'ingredients' column and store the output in 'cleaned_ingredients' column.
recipes_df['cleaned_ingredients'] = recipes_df['ingredients'].apply(clean_items)
recipes_df

,recipe_urls,recipe_name,ingredients,cleaned_ingredients
0,https://www.jamieoliver.com/recipes/duck-recip...,Roast duck with Marsala gravy,"['1 x 1.6kg whole duck', '2 heaped teaspoons C...","[duck, s chinese powder, clementine, bay leave..."
1,https://www.jamieoliver.com/recipes/vegetable-...,Best-ever Brussels sprouts,"['800 g Brussels sprouts', '2 higher-welfare C...","[brussels sprouts, sausages, sage, unsalted bu..."
2,https://www.jamieoliver.com/recipes/pasta-reci...,Beautiful courgette carbonara,"['6 medium green and yellow courgettes', '500 ...","[green yellow courgettes, penne, eggs, cream, ..."
3,https://www.jamieoliver.com/recipes/vegetable-...,Roasted black bean burgers,"['1½ red onions', '200 g mixed mushrooms', '10...","[red onions, mushrooms, rye bread, coriander, ..."
4,https://www.jamieoliver.com/recipes/chicken-re...,Chicken & tofu noodle soup,"['2 shallots', '2 cloves of garlic', '2 cm pie...","[shallots, cloves garlic, ginger, chicken thig..."
...,...,...,...,...
4642,https://www.allrecipes.com/recipe/213809/thai-...,Thai-Style Steamed Pumpkin Cake Recipe - Allre...,"[""1 (2 pound) pumpkin - peeled, seeded, and gr...","[pumpkin, s arrowroot powder, coconut milk, wh..."
4643,https://www.allrecipes.com/recipe/41751/delici...,Delicious Spicy Tomato Salad Recipe - Allrecip...,"[""1 apple - peeled, cored and sliced"",""1 chile...","[apple, chile pepper, salted peanuts, lemon, s..."
4644,https://www.allrecipes.com/recipe/264507/spicy...,Spicy Sriracha Meatballs Recipe - Allrecipes.com,"[""1 egg"",""1 lime, zested"",""1 tablespoon creamy...","[egg, lime, creamy peanut butter, o oil, mince..."
4645,https://www.allrecipes.com/recipe/264012/ajad-...,Ajad (Authentic Thai Cucumber Salad) Recipe - ...,"[""1 1/2 teaspoons salt"",""1/2 cup white vinegar...","[s salt, white egar, water, thai bird's eye ch..."


Now I notice that there are still some preprocessing steps to add. The 'preprocess_text' function reiterates the process of removing stopwords, but this time with the NLP package, which is more extensive. It also singularizes nouns, which helps standardize them, and lemmatizes. Lemmatizing is a method for reducing words to their base form.

In [10]:
p = inflect.engine()

def preprocess_text(text: str) -> str:
    """ Takes an ingredient and returns a processed version of that ingredient, without stopwords and after the
    singularization of nouns and lemmatization.

    Parameters: An ingredient or a string list of ingredients

    Returns: A processed ingredient or string list of ingredients
    """
    tokens = word_tokenize(text.lower())

    # Take out stopwords (reiterates process from earlier, this time with an NLP method).
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.isalnum() and token not in stop_words]

    # Singularize nouns.
    tokens = [p.singular_noun(token) or token for token in tokens]

    # Lemmatize words.
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return tokens

Previously, I removed measuring words, description words, and similar elements from the ingredients list. However, upon observation, it became clear that some of the ingredients which remained were not useful. To address this, I compiled a list, 'excluded_words,' containing words that didn't necessarily warrant complete removal from the DataFrame, but which I did not want to be the complete identifier for an ingredient. For instance, 'red chili paste' is a valuable ingredient, but 'red' is not. Thus, it is important to retain 'red' in the context of a phrase such as 'red chili paste,' while also preventing it from becoming a complete ingredient identifer.

In [11]:
excluded_words = ['red', 'per', 'cut', 'green', 'white', 'shredded', 'boneless', 'frozen',
                  'minced', 'baby', 'sweet', "hot", "italian", "pure", "black",
                  "yellow", "chinese", "dash", "thai", "orange", "vegetable",
                  "brown", "fillet", "spicy", "herb", "pickled", "mild", "packet",
                  "cream", "dark", "instant", "wild", "salad", "fat", "french",
                  "little", "square", "link", "unsweetened", "asian", "spring",
                  "bag", "jarred", "unsalted", "jar", "head", "coarse", "round",
                  "spice", "seasoning", "condensed", "warm", "golden", "seasoned",
                  "baking", "juice", "skin", "purple", "pastry", "salted", "mexican",
                  "halfsalt", "patak", "hard", "pureed", "chunk", "seed", "topping",
                  "ask", "sweetened", "sichuan", "liquid", "flat", "water", "boned",
                  "mixture", "sauce", "leg", "nonfat", "stone", "rack", "english",
                  "wedge", "wheat", "jumbo", "crispy", "kosher", "low", "jigger", "winter",
                  "fruit", "bitter", "peel", "tail", "seedless", "podded", "tube",
                  "paper", "boiled", "dipping", "crumbly", "granulated", "cracked",
                  "glass", "lank", "waxy", "spanish", "king", "spiced", "yong",
                  "deli", "ice", "outer", "field", "japanese", "guanciale", "zesty",
                  "evaporated", "heavy", "miniature", "crunchy", "see", "galangal",
                  "spun", "sour", "high", "shin", "fried", "dripping", "curried",
                  "salty", "clove", "simple", "breast", "crusty", "vine", "meltr", "compressed",
                  "angle", "responsibly", "cheesy", "bittersweet", "hp", "whiting",
                  "knife", 'deping', 'runny', 'sourced', 'gutted', 'hand', 'heated',
                  'flattened', 'cow', 'broad', 'ownr', 'brushed', 'rustic', 'meaty',
                  'bottom', 'defrosted', 'open', 'cets', 'sps', 'sprinkle', 'scale',
                  'spiral', 'third', 'crumb', 'short', 'easy', 'crystallized', 'heinzr',
                  'soured', 'eag', 'grain', 'multigrain', 'spray', 'silver', 'syrup',
                  'swansonr', 'overnight',  'ter', 'ature', 'kraft', 'ribbon', 'flake',
                  'kikkoman', 'available, ''wholegrain', 'virgin', 'half', 'pack', 'debd',
                  'colored',  'wholewheat', 'loaf', 'er', ' neck', 'including',  'sale',
                  'filling', 'super', 'strong', 'herby', 'Indian', 'creamy', 'zest', 'ear',
                  'boiling', 'roast', 'greek', 'candied', 'ced', 'plum', "sea", "heeng", "bap",
                  'meat'  'vegan', 'leaf', 'wholemeal', 'gen', 'sp', 'first', 'conned', 'gra',
                  'tablepsoon', 'Roasg', 'purchased', 'ped', 'find', 'bruised',  'bite',  'rell',
                   'scraped', 'passion', 'lite', 'middle',  'icola', 'uda', 'substitute', 'foot',
                  'interval',  'unripe',  'spectrumr', 'ing',  'soruces', 'san', 'button', 'rom',
                  'serstm', 'deded', 'tablespooons', 'lenghwise', 'rind', 'frond', 'tap', 'thigh',
                  'best', 'vital', 'body', 'commsed', 'puffed', 'sister', 'poug', 'tepid', 'john',
                  'le', 'vein', 'test', 'melg', 'five', 'solid', 'teapsoon', 'frond', 'knife', 'variety',
                   'original',  'grown', 'pick', 'without', 'even', 'buttery', 'breakfast', 'solid',
                  'sprinkle', 'food', 'easy', 'style', 'jar', 'breast', 'low', 'fat', 'equal', 'lenghwise',
                  'powder', 'sticky', 'peel', 'diagonal', 'portioned', 'method', 'brushed', 'Heinz',
                  'mushy', 'cellophane', 'bit', 'overnight', 'squashed', 'salty', 'available', 'oven',
                  'crumbling', 'find', 'lovely', 'instant', 'kept', 'low', 'warm', 'higher', 'cupboard',
                  'ear', 'semi', 'bag', 'intramuscular', 'rub', 'butterfly', 'produce', 'including',
                  'cottage', 'dip', 'way', 'got', 'bottom', 'sourced', 'field', 'outer', 'timing', 'centre',
                  'pack', 'mixture', 'weight', 'humane', 'Canadian', 'heirloom', 'creamy', 'blood',
                  'regular', 'per', 'test', 'together', 'omit', 'spicy', 'dripping', 'half', 'boned',
                  'deded', 'fat', 'five', 'one', 'welsh', 'patch', 'winter', 'get', 'dash', 'unwaxed',
                  'crusty', 'sparkling', 'teapsoon', 'paste', 'unsweetened', 'super', 'co', 'filling',
                  'defrosted', 'strong', 'American', 'substitute', 'jarred', 'mild', 'runner', 'wedge',
                  'baking', 'silken', 'eat', 'clear', 'teaspoon', 'pierced', 'head', 'ask', 'er',
                  'forced', 'stone', 'thai', 'english', 'drunken', 'butterflied', 'Mmexican', 'cracked',
                  'tablespoons', 'tablepsoon', 'dark', 'minced', 'broad', 'brown', 'spa', 'wild', 'waxy', 'virgin'
                 ]

Now I apply these changes to the DataFrame. I begin by creating 'recipes_df2', which is a duplicate of the 'recipes_df' DataFrame. I drop an unnecessary column (the 'ingredients' column, which contains the ingredients pre-processing), and rename the columns to be 'Links', 'Name', and 'Ingredient.'

Then, I create a 'recipes_temp' DataFrame which is a duplicate of 'recipes_df2,' to serve as the base for the final preprocessing steps. For each ingredient in every row of the 'Ingredient' column, I preprocess with 'preprocess_text'.

I then use the preprocessed 'Ingredient' column to create a new, more extensive ingredient column to replace it. I add all unigrams from each preprocessed ingredient, as well as the full phrase, to this new ingredient column, except for words/phrases included in the 'excluded_words' list.

I consciously made the choice to include both unigrams and phrases to the ingredient list in order to accomodate both one-word, and more detailed, searches. By including both options, the user will be able to search up 'onion,' and find all recipes which contain onion, *or* 'red onion,' and receive all recipes which contain red onion. This allows the user more flexibility and the option to input simpler or more complex requests, based on their preference.

I then drop the original 'Ingredient' column and retain this new 'Ingredient' column instead.

Lastly, I assign a copy of the 'recipes_temp' DataFrame to the name 'official_recipes_df.' This fully preprocessed DataFrame will be useful for the next stages of this project.

Note that I create duplicates of the DataFrame along the way in order to retain the DataFrame at various stages of preprocessing.

In [12]:
# Create 'recipes_df2,' which is a duplicate of 'recipes_df' but retains only the columns 'Links', 'Name',
# and 'Ingredient.'
recipes_df2 = recipes_df.copy()
recipes_df2 = recipes_df2.drop('ingredients', axis=1)
recipes_df2.rename(columns={'recipe_urls': 'Links', 'recipe_name': 'Name', 'cleaned_ingredients': 'Ingredient'}, inplace=True)

# Create 'recipes_temp', a duplicate of 'recipes_df2,' and then apply the final preprocessing steps and remove
# ingredients from the 'excluded_words' list.
recipes_temp = recipes_df2.copy()

# Instantiate 'new_list_of_lists' to contain the processed version of the ingredient lists.
new_list_of_lists = []
# Iterate over every row in the 'Ingredient' column.
for row_num in range(len(recipes_temp['Ingredient'])):
    curr_ingredient_list = recipes_temp['Ingredient'][row_num]
    new_ingredient_list = []
    # For every ingredient in the current ingredient list (row), apply the preprocess_text() function and remove
    # 'excluded_words.'
    for ingredient in curr_ingredient_list:
        preprocessed_ingredient = preprocess_text(ingredient)
        # Add in any unigrams in the phrase (i.e. the 'red' part of 'black bean')
        for pre_ingredient_sep in preprocessed_ingredient:
            # Add in the entire phrase, if it is not already in the dictionary (i.e. 'black bean')
            if pre_ingredient_sep not in excluded_words:
                new_ingredient_list.append(pre_ingredient_sep)
        preprocessed_ingredient = ' '.join(preprocessed_ingredient)
        if preprocessed_ingredient not in excluded_words:
           new_ingredient_list.append(preprocessed_ingredient)
    # Append the processed ingredient list.
    new_list_of_lists.append(new_ingredient_list)

# Remove the original 'Ingredient' column and replace it with the fully processed version.
recipes_temp['Preprocessed Ingredients'] = new_list_of_lists
recipes_temp = recipes_temp.drop('Ingredient', axis=1)
recipes_temp.rename(columns={'Preprocessed Ingredients': 'Ingredient'}, inplace=True)

# Create 'official_recipes_df,' the main DataFrame to be used going forward. It has the columns 'Links', 'Name', and 'Ingredient' to contain the URL,
# recipe name, and ingredients for each recipe. Though the 'Ingredient' column will later be removed, it is necessary to keep it for now in order to
# build the fourth column, which will contain the binary arrays associated with the recipes.
official_recipes_df = recipes_temp.copy()

# Creating Binary Arrays for All Recipes to be Used for Computing Cosine Similarity

The next step is to create binary arrays for every recipe in the DataFrame. Every array will contain 4647 elements, as this is the number of unique ingredients among all recipes. In every array, each index will correspond to a specific ingredient. The array associated with a particular recipe will have a 1 in the index associated with each of its ingredients, and a 0 otherwise.

As an example, consider a tomato feta salad. In the associated 4647-element array, the positions corresponding to tomato, feta, olive oil, and salt will be marked with 1, while all other positions will be marked with 0.

These arrays will be useful later, during the computation of cosine similarity.

The first step is to create a list of all the unique ingredients which appear at least once in any of the recipes in the DataFrame. The 'group_by_ingredient' function performs this step below when applied to 'recipes_df.' Afterwards, 'grouped_ingredients' is a dictionary. Each key in the dictionary corresponds to a unique ingredient.

In [13]:
def group_by_ingredient(recipes_df: 'pd.DataFrame') -> dict:
    """
    Takes the Recipes DataFrame and returns a dictionary. In this dictionary, each key is associated with a unique
    token ingredient. Each value contains an index.

    Parameters: The Recipes DataFrame

    Returns: A dictionary, for which each key is a unique ingredient token and the value is an index.
    """
    # Initialize the dictionary.
    grouped_ingredients = {}

    # Iterate over all rows of the DataFrame.
    index = 0
    for i in range(len(official_recipes_df)):
        ingredient_list = official_recipes_df['Ingredient'][i]
        for ingredient in ingredient_list:
            index = index + 1
            if ingredient:
                key = ingredient

            if key not in grouped_ingredients:
                grouped_ingredients[key] = [index]
            else:
                grouped_ingredients[key].append(index)

    # Exclude these words from frequency calculation
    for word in excluded_words:
        if word in grouped_ingredients:
            del grouped_ingredients[word]

    return grouped_ingredients

# Apply the 'group_by_ingredient' function to the original 'recipes_df.' The keys in the resulting
# 'grouped_ingredients' are the unique ingredient tokens.
grouped_ingredients = group_by_ingredient(recipes_df)

Now I prepare for creating the binary arrays associated with each recipe. I create the 'ingredient_index_dict' dictionary, which contains indices for keys and the associated ingredient for values. This dictionary will facilitate fast look-up of index for every unique ingredient later on.

I also build the 'create_array' function, which returns a list of binary arrays for a given list of indices. This function will be used in the next step to create a binary array for every recipe in the DataFrame. Later on, it will also be used to create a binary array to represent the user's ingredient specifications.

In [14]:
# Create ingredient_list, which contains all the unique ingredient tokens.
ingredient_list = list(grouped_ingredients.keys())

# Create a new DataFrame, ingredient_index_df, where column 1 is the index and column 2 is the ingredient.
# This way, every ingredient is associated with an index. It will be useful later during the creation of
# the binary arrays.
ingredient_index_df = pd.DataFrame({'Index': range(1, len(ingredient_list) + 1), 'Ingredient': ingredient_list})

# Now create a dictionary, 'ingredient_index_dict', for which the key is a unique ingredient and the value is
# the index. This will make index look-up fast.
ingredient_index_dict = dict(zip(ingredient_index_df['Ingredient'], ingredient_index_df['Index']))

def create_array(ingredient_indices: list) -> tuple:
    """
    Takes a list of indices and produces a binary array. This array is -elements long, and contains a 1 at every
    index contained in ingredient_indices, and a 0 otherwise.

    Parameters: A list of indices, each of which is associated with an ingredient.

    Returns: A binary array (list of integers) and the binary string (the string version of the binary array)
    """
    recipe_binary_string = ''
    # Iterate over every unique ingredient, in order. For each ingredient, check to see if it is in the given list,
    # ingredient_indices. If it is, append a 1 to 'recipe_binary_string'. Otherwise, append a 0.
    for j in range(1, len(ingredient_list)+1):
        if j in ingredient_indices:
            recipe_binary_string = recipe_binary_string + '1'
        else:
            recipe_binary_string = recipe_binary_string + '0'

    # Remove brackets and convert the string to a list of characters.
    binary_list = (recipe_binary_string[1:-1])

    # Convert each character to an integer.
    binary_array = [int(bit) for bit in binary_list]

    return binary_array, recipe_binary_string

Now create a list of binary arrays by applying the 'create_array' function. For each recipe, there is 'ingredient_indices,' a list of the indices associated with every ingredient in the recipe. Then, this 'ingredients_indices' list is used to create the binary array for the recipe.

After this, the 'array_list' will contain the binary arrays for every recipes in the 'official_recipes_df' DataFrame, in order.

I then create another column for 'official_recipes_df,' called 'Vector String', which contains this list of binary arrays.

In [15]:
# Initialize 'array_list' to contain the binary array list for each recipe, in the order they present in
# 'official_recipes_df'. Then, 'array_list_strings' contains the list of the string versions of the binary array.
# This will be useful later, because SQLite cannot comprehend a database with lists as elements.
array_list = []
array_list_strings = []

# Iterate over every recipe.
for i in range(len(official_recipes_df['Name'])):
    ingredient_indices = []
    # Create 'curr_ingredient_list' to contain the ingredient list for a particular recipe.
    curr_ingredient_list = official_recipes_df['Ingredient'][i]
    for ingredient in curr_ingredient_list:
        if ingredient in ingredient_index_dict:
            ingredient_index = ingredient_index_dict[ingredient]
        # Use 'ingredient_indices' to store the indices associated with the ingredients for a particular recipe.
        ingredient_indices.append(ingredient_index)

    # Get the binary array and binary array string associated with a particular recipe.
    binary_array, recipe_binary_string = create_array(ingredient_indices)

    # Append the binary array to 'array_list'.
    array_list.append(binary_array)
    array_list_strings.append(recipe_binary_string)

# Create a new column in the DataFrame which contains the string versions of the binary arrays.
official_recipes_df['Vector String'] = array_list_strings

# Connect to SQLite Database

Now I create and connect to a SQLite database, and insert the data from the 'official_recipes_df2' into this database. This will be necessary for the SQL querying that will be performed later.

Note that I have to use the DataFrame 'official_recipes_df2' for this, which is a copy of 'official_recipes_df' but without the 'Ingredient' column. Having this column was useful for creating the 'Vector String' column. However, it is no longer necessary and because it contains lists, it does not work with SQL, so I remove it. I also rename the vector column.

The 'official_recipes_df2' has the columns 'Links', 'Name', and 'Vector_String'. I then insert it into the SQLite database, calling it 'links_table.'

In [16]:
official_recipes_df2 = official_recipes_df.copy()

# Rename the vector to make it compatible.
official_recipes_df2.rename(columns={'Vector String': 'Vector_String'}, inplace=True)

# Drop this column as it contains lists, and SQLite cannot handle this format.
official_recipes_df2.drop('Ingredient', axis=1, inplace=True)

In [17]:
#Create and connect to the SQLite database
conn = sqlite3.connect('links.db')
c = conn.cursor()

#Insert the data from the recipes dataframe into the SQLite database
official_recipes_df2.to_sql('links_table', conn, if_exists='replace', index=False)

# Commit changes and close the connection
conn.commit()
conn.close()

# Helper Functions

The function below, 'get_similar_recipe,' calculates cosine similarity. This will be necessary later, when sorting a subset of recipes by their similarity to the user's specifications.  

In [18]:
def get_similar_recipe(links: list, desired_array: list, array_list: list) -> pd.DataFrame:
    """
    There is a subset of recipes which fits the mandatory requirements (contain all mandatory ingredients and exclude
    all unwanted ingredients). This function takes the links for these recipes. Additionally, it takes
    'desired array' (the binary array associated with all the user's requested ingredients) and 'array_list'
    (list of binary arrays associated with the recipes in the subset). It performs a Cosine Similarity between
    the 'desired_array' and 'array_list', thus producing a DataFrame sorted in descending order in terms of Cosine
    Similarity. In other words, it produces a list of links, sorted based on their similarity to the user's
    specifications. Since all the recipes returned fit the mandatory requirements, the order of the recipes is
    based on whether the recipe also has the optional ingredients. Recipes which contain the user's specified
    optional ingredients, and as few extra ingredients as possible, are at the top of the list.

    Parameters: List of links, list of integers (binary array), and list of binary arrays

    Returns: DataFrame of links to recipes, sorted in descending order with regards to their Cosine similarity
    """
    # Converts the desired array (array associated with user ingredient specifications) to a numPy aray. Does the
    # same for all the binary arrays associated with all the subset of recipes which fit the mandatory requirements.
    target_array = np.array(desired_array)
    arrays_in_dataframe = np.array(array_list)

    # Calculate cosine similarity between them.
    cosine_similarities = cosine_similarity([target_array], arrays_in_dataframe)

    # Create a DataFrame with a Links column and a Cosine Similarity score column.
    df2 = pd.DataFrame({
        'Links': links,
        'Cosine_Similarity': cosine_similarities[0]
    })

    # Sort the DataFrame based on cosine similarity in descending order.
    df2_sorted = df2.sort_values(by='Cosine_Similarity', ascending=False)

    return (df2_sorted)

The following function, 'singularize,' will be useful in the next step, because it will be used to singularize the nouns contained in the user input.

In [19]:
def singularize(word: str) -> str:
    """Singularizes all nouns in the given string

    Parameter: String

    Returns: String for which all nouns are now in the singular form
    """
    if word:
        p = inflect.engine()
        singular_form = p.singular_noun(word)
        if singular_form:
            return singular_form
    return word

# Recipe Suggestor

Below is the Recipe Suggestor.

The program asks the user for mandatory ingredients (ingredients that *must* be in the recipe), unwanted ingredients (ingredients that *cannot* be in the recipe), and optional ingredients. All recipes returned will contain all mandatory ingredients, and none of the unwanted ingredients. They may contain some optional ingredients. The recipes at the top of the list are most likely to contain the optional ingredients, with as few added (unspecified) ingredients as possible. This makes it as likely as possible that the user will find a recipe which they can make at home, based on their ingredient specifications.

In [35]:
print("Welcome to Recipe Suggestor! Please use lists of foods in your responses, with no additional words added. For example, chicken, sugar, broccoli. It is fine to leave some fields blank.")

# Question 1: In the case the user chooses foods they definitely want / must all be in the recipe
mandatory_ingredients_string = input("Please enter the ingredients that you must have in the recipe (comma-separated list): ")

# Question 2: In the case the user wants to avoid certain ingredients
unwanted_ingredients_string = input("Please enter any ingredients that you do NOT want to be in the recipe (comma-separated list): ")

# Question 3: For the user to input ingredients they have on hand, which may optionally be included
optional_ingredients_string = input("Please enter any ingredients that you have on hand that you wouldn't mind being in the recipe (comma-separated list): ")

# After the user responds to the questions, the responses are contained in 'mandatory_ingredients_string',
# 'unwanted_ingredients_string', and 'optional_ingredients_string'. Below, I convert the strings to lower case,
# convert them to lists, and singularize every ingredient in the list. The end result are lists called
# 'mandatory_ingredients', 'unwanted_ingredients', and 'optional_ingredients'.

mandatory_ingredients = mandatory_ingredients_string.lower()
mandatory_ingredients = mandatory_ingredients.split(', ')
mandatory_ingredients = [singularize(word) for word in mandatory_ingredients]

unwanted_ingredients = unwanted_ingredients_string.lower()
unwanted_ingredients = unwanted_ingredients.split(', ')
unwanted_ingredients = [singularize(word) for word in unwanted_ingredients]

optional_ingredients = optional_ingredients_string.lower()
optional_ingredients = optional_ingredients.split(', ')
optional_ingredients = [singularize(word) for word in optional_ingredients]

# Initialize lists to contain the indices of each ingredient, as represented in ingredient_index_dict. This will
# be important later, when we search for recipes which include the ingredients requested (and do not include the
# unwanted ingredients).

# The list 'mandatory_indices' will contain the indices associated with the mandatory ingredients,
# 'unwanted_ingredients' will contain the indices associated with the unwanted ingredients, and 'optional_indices'
# will contain the indices associated with the optional ingredients, IF we have recipes on file which contain the
# ingredient. I check to see if there is a recipe on file with the ingredient with the line, 'if ingredient in
# 'ingredient_dict.' This is because 'ingredient_dict' contains all ingredients for which there exists an
# associated recipe in the data base.

#Then, 'present_ingredients' will contain the ingredients from 'mandatory_ingredients' and 'optional_ingredients'
# for which we do have recipes, and 'absent_ingredients' will contain the ingredients from these two lists for which we do NOT have any recipes.

mandatory_indices = []
unwanted_indices = []
optional_indices = []
total_indices_desired = []
present_ingredients = []
absent_ingredients = []

for ingredient in mandatory_ingredients:
    if ingredient in ingredient_index_dict:
        index_needed = ingredient_index_dict[ingredient]
        mandatory_indices.append(index_needed)
        present_ingredients.append(ingredient)
    else:
        absent_ingredients.append(ingredient)

for ingredient in unwanted_ingredients:
    if ingredient in ingredient_index_dict:
        index_needed = ingredient_index_dict[ingredient]
        unwanted_indices.append(index_needed)

for ingredient in optional_ingredients:
    if ingredient in ingredient_index_dict:
        index_needed = ingredient_index_dict[ingredient]
        optional_indices.append(index_needed)

# Create a 'total_indices_desired' to represent the indices for all the ingredients requested by the user. If
# there are any 'optional' ingredients requested, merge the 'mandatory' and 'optional' ingredients together.

if optional_indices != []:
    total_indices_desired = mandatory_indices + optional_indices
else:
    total_indices_desired = mandatory_indices

# Use helper function 'create_array' to store the vector associated with the ingredients requested. We're working
# with a 4647-element binary array where each index corresponds to a specific food. For each requested food,
# there's a '1' at its corresponding index, and '0' at all other indices.
# After, 'desired_array' will contain the array, and 'desired_binary_string' will contain the string version of it.
desired_array, desired_binary_string = create_array(total_indices_desired)

# The next step is to gather the links for the subset of recipes which fit the initial mandatory requirements (recipes which include all mandatory
# ingredients and exclude all unwanted ingredients). Handle every combination of cases:

# Case 1: If the user requests an ingredient as both mandatory (or optional) and unwanted, this creates a conflict,
# so the user is instructed to try again.
for index in unwanted_indices:
    if index in total_indices_desired:
        print("Please check your input. It is not possible to request the same ingredient as both mandatory and unwanted.")

# Case 2: If there are no recipes with any of the requested nor unwanted ingredients, the user is instructed
# to try again.
if total_indices_desired == [] and unwanted_indices == []:
    print("Unfortunately, we could not find any recipes with those ingredients. Check spelling and try again.")
elif len(mandatory_ingredients) != len(mandatory_indices):
  print("Unfortunately, we could not find any recipes with all of the mandatory ingredients. Check spelling and try again.")

# If there is at least one mandatory, optional, or unwanted ingredient and no conflicts, the program proceeds.
else:
    # Case 3: There is an optional ingredient requested, but no mandatory or unwanted ingredients.
    if mandatory_indices == [] and unwanted_indices == []:
        sql_query = f'''
                SELECT Links
                FROM links_table
               '''

    # Case 4: There are unwanted ingredients specified but no mandatory ingredients.
    # Note that the RegExpression in the Where condition checks if the specific bits at positions associated with
    # the unwanted ingredients in Vector_String are all '0'. Loops through every relevant index and checks.
    # Returns the links associated with recipes for which the condition is satisfied, i.e. the links to recipes
    # which do contain the mandatory ingredients.
    elif mandatory_indices == [] and unwanted_indices != []:
        sql_query = f'''
        SELECT Links
        FROM links_table
        WHERE {' AND '.join([f"SUBSTR(Vector_String, {ingredient_index_dict[ingredient]}, 1) = '0'" for ingredient in unwanted_ingredients if ingredient])}
    '''

    # Case 5: There are either mandatory or optional ingredients specified, but no unwanted ingredients.
    # Note that the RegExpression in the Where condition checks if the specific bits at positions associated with
    # the mandatory ingredients in Vector_String are all '1'. Returns links for which this condition is satisfied,
    # i.e. the links to recipes which do contain the mandatory ingredients.
    elif total_indices_desired != [] and unwanted_indices == []:
        # Construct the SQL query dynamically
        sql_query = f'''
            SELECT Links
            FROM links_table
            WHERE {' AND '.join([f"SUBSTR(Vector_String, {index}, 1) = '1'" for index in total_indices_desired])}
        '''

    else:
        # There are unwanted ingredients specified, as well as either mandatory or optional ingredients.
        # Returns links for which the recipe contains the mandatory ingredients and does not contain the unwanted
        # ingredients.
        sql_query = f'''
            SELECT Links
            FROM links_table
            WHERE {' AND '.join([f"SUBSTR(Vector_String, {index}, 1) = '1'" for index in mandatory_indices])}
            {' AND ' if unwanted_ingredients else ''}{' AND '.join([f"SUBSTR(Vector_String, {ingredient_index_dict[ingredient]}, 1) = '0'" for ingredient in unwanted_ingredients if ingredient])}
           '''

    conn = sqlite3.connect('links.db')

    # Create a cursor to interact with the database
    c = conn.cursor()

    # Execute the SQL query (the correct query for the case)
    c.execute(sql_query)

    # Fetch the results of the query and store them in 'links_to_find.' This list contains the links of the recipes
    # which the SQL query found. Specifically, thery are links to recipes which contain the mandatory ingredients
    # and exclude the unwanted ingredients. The next step is to find recipes which also have the optional ingredients,
    # and as few extra (unspecified) ingredients as possible.
    links_to_find = c.fetchall()

    # If links_to_find is empty, it means there are no recipes which meet the specifications.
    if links_to_find == []:
        print("Unfortunately, we could not find any recipes with all of required ingredients. Try again with fewer required ingredients, and consider adding some of the ingredients to the optional section instead.")

    # If there is at least one recipe, then proceed. This step is for using cosine similarity to find the recipes
    # that contain the most optional ingredients, and the fewest unspecified ingredients.
    else:

        # The purpose of this query is to find the binary vector associated with the given link. This requires
        # accessing the links_table database, and selecting the vector string associated with the link.
        sql_query2 = f'''
                  SELECT Vector_String
                  FROM links_table
                  WHERE Links LIKE ?;
                 '''

        # Initialize the list to be used to store the binary arrays associated with the links.
        list_of_arrays = []

        # For every recipe link, execute a SQL query to fetch the binary array.
        for link in links_to_find:
            c.execute(sql_query2, ('%' + link[0] + '%',))
            result = c.fetchall()

            # Ensures that only the first row is appended.
            if result:
                row_num = 0
                for row in result:
                    if row_num == 0:
                        list_of_arrays.append(row[0])
                        row_num = row_num + 1
                    else:
                        break
            else:
                # Handle the case where no result is found for the link.
                # For now, append None to maintain the length consistency.
                list_of_arrays.append(None)

        # The list of binary arrays needs to be edited first by converting each character to an integer.
        # Initialize a list to store the edited binary arrays
        edited_list_of_arrays = []

        for array in list_of_arrays:
            binary_list2 = (array[1:-1])

            # Convert each character to an integer
            binary_array2 = [int(bit) for bit in binary_list2]
            # Append each edited binary array to 'edited_list_of_arrays'.
            edited_list_of_arrays.append(binary_array2)
            edited_list_of_arrays

        # The 'edited_list_of_arrays', contains the binary arrays associated with all the recipes which adhere to
        # the user specifications (i.e., they contain the mandatory ingredients and exclude the unwanted ingredients).
        # Using the 'get_similar_recipe' helper function, the code computes cosine similarity between 'desired_array'
        # (the binary array which represent the user's specified mandatory and optional ingredients) and
        # 'edited_list_of_arrays'. The outcome is a list of recipe links sorted by cosine similarity, with those at
        # the top most closely aligning with the user's preferences. These recipes are most likely to contain the
        # optional ingredients on top of the other specifications, and as few extra ingredients as possible.

        df_sorted = get_similar_recipe(links_to_find, desired_array, edited_list_of_arrays)

        # Now that 'edited_list_of_arrays' contains the sorted list of recipes, I prepare the output. There needs
        # to be specific messages printed based on the case. In all of the following cases, there is at least one
        # recipe suggested to the user.

        # In the case that there were no recipes which contained some or all of the optional ingredients.
        if absent_ingredients != [] and absent_ingredients != ['']:
            if len(absent_ingredients) == 1:
                print("Unfortunately, we could not find recipes with:", absent_ingredients[0])
            elif len(absent_ingredients) == 2:
                print(f"Unfortunately, we could not find recipes with: ", absent_ingredients[0], "or", absent_ingredients[1], ".")
            else:
                print("Unfortunately, we could not find recipes with: ", ', '.join(absent_ingredients[:-1]), f"or {absent_ingredients[-1]}.")

        # In the case that there were recipes found that contain all of the mandatory and optional ingredients.
        if mandatory_ingredients != [''] and absent_ingredients == []:
            print("We were able to find recipes with the ingredients requested.")
            print("They contain ", present_ingredients)

        # If there were recipes found that missed some of the optional ingredients but not all, explain which of the
        # optional ingredients were found in recipes.
        if mandatory_ingredients != [''] and absent_ingredients != [] and present_ingredients != []:
            if len(present_ingredients) == 1:
                print("However, we were able to find recipes with the other requested ingredient. They contain", present_ingredients[0])
            elif len(present_ingredients) == 2:
                print(f"However, we were able to find recipes with the other requested ingredients. They contain {present_ingredients[0]} and {present_ingredients[1]}.")
            else:
                print("However, we were able to find recipes with the other requested ingredients. They contain", ', '.join(present_ingredients[:-1]), f"and {present_ingredients[-1]}.")

        # If there were no mandatory ingredients requested.
        if mandatory_ingredients == []:
            print("We were able to find some recipes with these ingredients.")
        if unwanted_ingredients != ['']:
            if len(unwanted_ingredients) == 1:
                print("We found the following recipes. They do not contain ", unwanted_ingredients[0])
            elif len(unwanted_ingredients) == 2:
                print(f"We found the following recipes. They do not contain {unwanted_ingredients[0]} or {unwanted_ingredients[1]}.")
            else:
                print("We found the following recipes. They do not contain ", ', '.join(unwanted_ingredients[:-1]), f"or {unwanted_ingredients[-1]}.")

        if optional_ingredients != ['']:
            print("We found the following recipes. The recipes on the top are most likely to contain the optional ingredients, with as few extra ingredients as possible.")
        print("Here are their links:")


        # Print out the sorted list of recipe links.
        for link_tuple in df_sorted['Links']:
            print(link_tuple[0])

        # Close the cursor and the connection
        conn.close()


Welcome to Recipe Suggestor! Please use lists of foods in your responses, with no additional words added. For example, chicken, sugar, broccoli. It is fine to leave some fields blank.
Please enter the ingredients that you must have in the recipe (comma-separated list): orange, pasta
Please enter any ingredients that you do NOT want to be in the recipe (comma-separated list): 
Please enter any ingredients that you have on hand that you wouldn't mind being in the recipe (comma-separated list): 
Unfortunately, we could not find any recipes with all of the mandatory ingredients. Check spelling and try again.
